## Group 7
##### Alec Arreola-Coburn, Cici Liu, Philip McCabe, Rishabh Shyamsunder, Elena Wu-Ratner

***

### Chapter 5, Slide 7 - The BMC Transshipment Problem

    Supply 500 > Demand 480, Inflow - Outflow >= Demand/Supply
   
    Decision variables: Xij for # of shipments from i to j
    Objective Function: MIN 30X12 + 40X14 + 50X23 + ...
                      : MIN SUM(Cij x Xij)
    S.T:
        Newark: -X12 - X14 >= -200
        Boston: X12 - X23 >= 100
        Columbus: X23 + X53 - X35 >= 60
        Richmond: X14 + X74 + X54 >= 80
        Atlanta: X75 + X65 + X35 - (X53 + X56 + X54) >= 170
        Mobile: X56 + X76 - X65 >= 70
        J'ville: -X76 - X75 - X74 >= -300

In [1]:
# Importing gurobi
from gurobipy import *

In [2]:
nodes = 7

# Creating multidict of paths, cost, and a dict of inflows to use later
paths, cost = multidict(
{(1,2):30,
(1,4):40,
(2,3):50,
(3,5):35,
(5,3):40,
(5,4):30,
(5,6):35,
(6,5):25,
(7,6):50,
(7,5):45,
(7,4):50})

inflows = {
    (1):200,
    (7):300,
    (2):-100,
    (3):-60,
    (4):-80,
    (5):-170,
    (6):-70,
}

In [3]:
# Creating the model
t = Model("Transshipment")

Using license file c:\gurobi\gurobi.lic
Academic license - for non-commercial use only


In [4]:
# Adding variable to model
route = t.addVars(paths, obj=cost, name='route')

In [5]:
# Adding constraints to model based on each node and the assigned inflow amount
for i in range(1, nodes+1):
    t.addConstr(sum(route[i,j] for i,j in paths.select(i,'*'))
               -sum(route[j,i] for j,i in paths.select('*',i))
                <= inflows[i])

In [6]:
t.update()

In [7]:
t.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 7 rows, 11 columns and 22 nonzeros
Model fingerprint: 0xf91af075
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+03   4.750000e+01   0.000000e+00      0s
       6    2.2350000e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds
Optimal objective  2.235000000e+04


In [8]:
# Printing the results in a clearer way
print('The path with the least cost is:')
for i,j in paths:
    if(route[i,j].x > 0):
        print(i,'-', j, ":", route[i,j].x)
print('Total Cost: ' + str(t.objVal))
print()

The path with the least cost is:
1 - 2 : 120.0
1 - 4 : 80.0
2 - 3 : 20.0
5 - 3 : 40.0
7 - 6 : 70.0
7 - 5 : 210.0
Total Cost: 22350.0



***

### Chapter 5, Slide 17 - Coal Bank Hollow Recycling Problem

    Supply 190 > Demand 150, Inflow - Outflow >= Demand/Supply
    
    Decision Varaibles: Xij for tons of products from i to j
    Objective Function: MIN 13X15 + 12X16 + 11X25 + 13X26...
                      : MIN SUM(Cij x Xij)
    S.T:
        Node 1: -X15 - X16 >= -70
        Node 2: -X25 - X26 >= -50
        Node 3: -X35 - X36 >= -30
        Node 4: -X45 - X46 >= -40
        Node 5: 0.9X15 + 0.8X25 + 0.95X35 + 0.75X45 – (X57 + X58 + X59) >= 0
        Node 6: 0.85X16 + 0.85X26 _ 0.9X36 + 0.85X46 - (X67 + X68 + X69) >= 0
        Node 7: 0.95X57 + 0.9X67 >= 60
        Node 8: 0.9X58 + 0.95X68 >= 40
        Node 9: 0.9X59 + 0.95X69 >= 50

In [9]:
nodes = 9

# Creating three lists for each "area" the paper goes through
origin = [1, 2, 3, 4]
trans = [5, 6]
demand = [7, 8, 9]

# Creating multidict for arcs and cost, and dicts for inflows and inflow percentages
arcs, cost = multidict(
{(1,5):13,
(1,6):12,   
(2,5):11,
(2,6):13,
(3,5):9,
(3,6):10,
(4,5):13,
(4,6):14,
(5,7):5,
(5,8):6,
(5,9):8,
(6,7):6,
(6,8):8,
(6,9):7})

inflows = {
    (1):70,
    (2):50,
    (3):30,
    (4):40,
    (5):0,
    (6):0,
    (7):60,
    (8):40,
    (9):50
}

percentage = {
(1,5):0.9,
(1,6):0.85,   
(2,5):0.8,
(2,6):0.85,
(3,5):0.95,
(3,6):0.9,
(4,5):0.75,
(4,6):0.85,
(5,7):0.95,
(5,8):0.9,
(5,9):0.9,
(6,7):0.9,
(6,8):0.95,
(6,9):0.95
}

In [10]:
# Creating model
r = Model('Recycling')

In [11]:
# Adding variables to the model
flow = r.addVars(arcs, obj=cost, name='flow')

In [12]:
# Adding constraints for the origin nodes
for i in origin:
    r.addConstr(sum(flow[i,j] for i,j in arcs.select(i,'*'))
                <= inflows[i])

In [13]:
# Adding constraints for the middle nodes
for i in trans:
    r.addConstr(sum(percentage[j,i]*flow[j,i] for j,i in arcs.select('*',i))
                -sum(flow[i,j] for i,j in arcs.select(i,'*'))
                >= 0)

In [14]:
# Adding constraints for the destination nodes
for i in demand:
    r.addConstr(sum(percentage[j,i]*flow[j,i] for j,i in arcs.select('*',i))
                -sum(flow[i,j] for i,j in arcs.select(i,'*'))
                >= inflows[i])

In [15]:
r.update()

In [16]:
r.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 9 rows, 14 columns and 28 nonzeros
Model fingerprint: 0x53185c80
Coefficient statistics:
  Matrix range     [8e-01, 1e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 7e+01]
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.500000e+02   0.000000e+00      0s
      11    3.1494559e+03   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds
Optimal objective  3.149455911e+03


In [17]:
# Pringting the path of the least cost
print('The path with the least cost is:')
for i,j in arcs:
    if(flow[i,j].x > 0):
        print(i, '-',j, ":", flow[i,j].x)
print('Total Cost: ' + str(r.objVal))
print()

The path with the least cost is:
1 - 5 : 43.44704353476282
1 - 6 : 26.55295646523718
2 - 5 : 50.0
3 - 5 : 30.0
4 - 6 : 35.366548178725665
5 - 7 : 63.15789473684211
5 - 8 : 44.44444444444444
6 - 9 : 52.631578947368425
Total Cost: 3149.4559110193786

